# Project Proposal: NYC 311 Call Data Analysis

### Introduction
Whenever people relocate to a new city or want to relocate to a different part within the same city, a number of factors influence their choice of neighborhood. Some examples are:
* Housing/rental prices.
* Crime rate in the area.
* Access to public transport.
* Condition of public services e.g. street lights, sewers, etc.
* Noise levels in the area (noise complaints, proximity to railway lines / airports).
* Distance from one's workplace.
* Distance from the airport (for frequent flyers).
* Quality of educational institutes in the area.

I propose building an online tool where:
* People can compare how different neighbourhoods within a city compare, based on criteria like the ones above. 
Both overall score, trends over time and future projections will be provided.
* Based on user input of how important each factor is to the user, a ranking of the different areas will be provided to the user.

NYC is particularly attractive city for the analysis, as it has large amounts of open data available. 
Some examples are:
* [311 Call Data](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9)
* [Taxi Trip Data](http://chriswhong.com/open-data/foil_nyc_taxi/)
* [Bike Sharing](http://www.citibikenyc.com/system-data)
* [NYC Restaurant Grades](http://www1.nyc.gov/site/doh/services/restaurant-grades.page)

A comprehensive listing of datasets can be found at [NYC Open Data](https://nycopendata.socrata.com/).

### Vision:
Provide an online interactive tool for NYC where:
* A listing of various factors representing the condition of an area is provided: e.g. safety, noise levels, etc.
* The user selects the relative importance of each factor: very important, important, nice-to-have, don't care.
* Based on the user's selection, a map is displayed, where different areas in the city are color-filled based on how well they meet the users criteria. A ranked list of the areas will also be provided.
* In addition, the user will be able to see trends of various factors for a particular area including future projections.
* In addition, user queries will be recorded to understand what factors are prioritized by people.
    
### Goal for the Semi-Final Challenge:
Realizing the above vision will require working with multiple datasets and conducting detailed analysis and model building. 
However, for the semifinal round, I am performing a limited analysis of NYC 311 dataset as a proof-of-concept.

The steps I followed are:

### Data Gathering:
NYC 311 data can be downloaded from [this](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9) site. The data is accessible via API, however, you need to apply for a key first, which wasn't feasible, given the timeframe of the challenge. Hence, I chose to directly download the data as a csv file. The file was ~6.8GB in size.

### Data Exploration and Cleaning:
I first started with a small subset of the data. I looked at the columns available, the kind of values in them, the set of unique values and their range.  

The zips needed some cleaning - some were only 2 digits, some do not belong to NY and some are plain bad: e.g. 'AIRPORT', 'IDK'. While in my final project I would like to carry out the analysis at the level of zips / blocks, for the current study, I stuck to the borough level.  

Based on the above choice, I selected a subset of the columns and then used Windows power shell to write out a new csv file that contained only the selected columns. This brought down the dataset size from ~6.8GB to ~1.32GB.

### Analysis:
In this proof-of-concept, I only performed a simple analysis:

First, I looked at the range of the date column: It was from '2010-01-01 00:00:00' to '2016-04-15 02:47:31'.
Since we have incomplete data for 2016, I filtered out values for 2016. I also added the year and month as columns to the dataframe.

Next, I looked at all the unique values of complaint type. There were 247 different values. I decided to focus on the top 25 complaints that accounted for 73.6% of the total complaints. Further, I grouped the top 25 complaints into categories as follows:

* **Heating and Electric**: ['HEATING', 'HEAT/HOT WATER', 'ELECTRIC' ]
* **Plumbing and Construction**: ['PLUMBING', 'GENERAL CONSTRUCTION', 'PAINT - PLASTER', 'NONCONST', 'General Construction/Plumbing', 'Building/Use']
* **Streets, Traffic and Parking**: ['Street Light Condition', 'Street Condition', 'Blocked Driveway', 'Illegal Parking', 'Traffic Signal Condition', 'Damaged Tree', 'Broken Muni Meter']
* **Water and Hygiene**: ['Water System', 'Sewer', 'Dirty Conditions', 'Sanitation Condition', 'UNSANITARY CONDITION', 'Rodent']
* **Noise**: ['Noise', 'Noise - Commercial', 'Noise - Street/Sidewalk']

(Assumption: 'NONCONST' is a construction related complaint - did not have time to investigate).

I created a new column 'Complaint_category' that captured this mapping. While implementing the final project, this step will require additional user research / local knowledge to ascertain that the groupings make sense. 

Next, I looked at the unique borough values:
'BROOKLYN', 'QUEENS', 'BRONX', 'MANHATTAN', 'STATEN ISLAND', 'Unspecified'.
Further analysis is required for rows with 'Unspecified' boroughs. For now, I decided to drop these rows.

Finally, I selected the appropriate data and created pivot tables (each with a different arrangement of data) and used the data for plotting. Specifically, I created two sets of plots: 

* One with a 'complaint type' focus, where one can see the values and trends in the number of complaints by year, for each borough for each complaint type. This is useful if the user is particular interested in making a choice based on a particular complaint type.

* The second plot has a 'borough' focus and shows the values and trends in various complaint types by year. This is useful if the user wants to see how the area is performing on various fronts.

Note: Even though absolute numbers are important, one cannot directly compare areas based on absolute numbers, since each has a different total area and different population density. For future analysis, these numbers should be normalized by the appropriate denominator to make a fair comparison. e.g. For the category 'Plumbing and Construction' one should normalize by the a factor representing the number of dwellings in the area: the population of the area could be a good proxy. One could also argue that the incident count per unit area is another good way of normalization. These can then be plotted on a map.

### Next Steps:
Next steps that can be taken are:
* Clean up zips data.
* Perform a finer binning based on zips / blocks.
* Investigate if the grouping of categories needs refinement.
* Investigate apropriate way of normalization of data.
* Plot data on a map.
* Pull data from additional sources.
* Investigate best way to combine factors to create an overall score.
* Port to Heroku - build in interactivity.
* Build in components to capture and analyze user preferences.
* Marketing of tool -make the tool a basis for dialogue around city services.


In [1]:
import numpy as np
import pandas as pd

pd.set_option('precision',10)

In [13]:
calls1 = pd.read_csv('./311_truncated_file_1.csv')

In [14]:
calls1[:3]

,created_date,Agency,Borough,complaint_type,incident_zip,City,Latitude,Longitude
0,NaN,DOB,BROOKLYN,NaN,NaN,BROOKLYN,40.6780042139,-73.9704404520
1,NaN,DOB,STATEN ISLAND,NaN,NaN,STATEN ISLAND,40.5914345421,-74.1647882378
2,NaN,DOB,QUEENS,NaN,NaN,Oakland Gardens,40.7476796746,-73.7594756840


In [19]:
calls1 = calls1.drop(['created_date', 'complaint_type', 'incident_zip'],axis=1)

In [20]:
calls1[:3]

,Agency,Borough,City,Latitude,Longitude
0,DOB,BROOKLYN,BROOKLYN,40.6780042139,-73.9704404520
1,DOB,STATEN ISLAND,STATEN ISLAND,40.5914345421,-74.1647882378
2,DOB,QUEENS,Oakland Gardens,40.7476796746,-73.7594756840


In [22]:
calls1.shape

(11122188, 5)

In [21]:
calls2 = pd.read_csv('./311_truncated_file_2.csv', parse_dates=['Created Date'])

C:\Users\Mario\AppData\Local\Enthought\Canopy\User\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
calls2.shape

(11122188, 3)

In [24]:
calls2[:3]

,Created Date,Complaint Type,Incident Zip
0,2013-08-09 15:01:09,Building/Use,11238
1,2013-08-09 14:44:28,Special Enforcement,10314
2,2013-08-09 17:35:45,Building/Use,11364


In [27]:
calls = pd.concat([calls2, calls1], axis=1)

In [28]:
calls.shape

(11122188, 8)

In [29]:
calls[:3]

,Created Date,Complaint Type,Incident Zip,Agency,Borough,City,Latitude,Longitude
0,2013-08-09 15:01:09,Building/Use,11238,DOB,BROOKLYN,BROOKLYN,40.6780042139,-73.9704404520
1,2013-08-09 14:44:28,Special Enforcement,10314,DOB,STATEN ISLAND,STATEN ISLAND,40.5914345421,-74.1647882378
2,2013-08-09 17:35:45,Building/Use,11364,DOB,QUEENS,Oakland Gardens,40.7476796746,-73.7594756840


In [30]:
calls.to_csv('311_truncated_data.csv')

In [31]:
sorted(calls['Borough'].unique())

['BRONX', 'BROOKLYN', 'MANHATTAN', 'QUEENS', 'STATEN ISLAND', 'Unspecified']

In [36]:
sorted(calls['Incident Zip'].unique())

[nan,
 0.0,
 83.0,
 7083.0,
 7086.0,
 7114.0,
 7201.0,
 7974.0,
 10000.0,
 10001.0,
 10002.0,
 10003.0,
 10004.0,
 10005.0,
 10006.0,
 10007.0,
 10008.0,
 10009.0,
 10010.0,
 10011.0,
 10012.0,
 10013.0,
 10014.0,
 10016.0,
 10017.0,
 10018.0,
 10019.0,
 10020.0,
 10021.0,
 10022.0,
 10023.0,
 10024.0,
 10025.0,
 10026.0,
 10027.0,
 10028.0,
 10029.0,
 10030.0,
 10031.0,
 10032.0,
 10033.0,
 10034.0,
 10035.0,
 10036.0,
 10037.0,
 10038.0,
 10039.0,
 10040.0,
 10044.0,
 10048.0,
 10065.0,
 10069.0,
 10075.0,
 10103.0,
 10111.0,
 10119.0,
 10128.0,
 10129.0,
 10162.0,
 10280.0,
 10281.0,
 10282.0,
 10301.0,
 10302.0,
 10303.0,
 10304.0,
 10305.0,
 10306.0,
 10307.0,
 10308.0,
 10309.0,
 10310.0,
 10312.0,
 10314.0,
 10451.0,
 10452.0,
 10453.0,
 10454.0,
 10455.0,
 10456.0,
 10457.0,
 10458.0,
 10459.0,
 10460.0,
 10461.0,
 10462.0,
 10463.0,
 10464.0,
 10465.0,
 10466.0,
 10467.0,
 10468.0,
 10469.0,
 10470.0,
 10471.0,
 10472.0,
 10473.0,
 10474.0,
 10475.0,
 10550.0,
 10553.0,
 10801

In [41]:
sorted(calls['Incident Zip'].unique())[-50:]

['98124',
 '98130',
 '98188',
 '98206',
 '98230',
 '98335',
 '99999',
 '999999',
 '?',
 'AIRPORT',
 'ANONYMOUS',
 "DON'T KNOW",
 'IDK',
 'INDIAN WEL',
 'JFK',
 'L4W 4Y5',
 'L7R1H2',
 'L8N1E5',
 'MEXICO7500',
 'MN55439',
 'N//A',
 'NEW JERSEY',
 'NEW YORK A',
 'NEWARK',
 'NEWARK AIR',
 'NEWARK INT',
 'NJ',
 'NJ 07114',
 'NJ 07201',
 'NO CLUE',
 'NO IDEA',
 'NONE',
 'NOT KNOWN',
 'NOT SURE',
 'NTY',
 'NY',
 'NY 10010-3',
 'NYC',
 'OOOOO',
 'PE60ND',
 'R3B0C9',
 'SS',
 'UNK',
 'UNKNOWN',
 'UNSURE',
 'X',
 'XXX',
 'XXXXX',
 'XXXXXXXXXX',
 'ZIP CODE']

In [46]:
calls['Created Date'].min()

Timestamp('2010-01-01 00:00:00')

In [45]:
calls['Created Date'].max()

Timestamp('2016-04-15 02:47:31')

In [57]:
calls1015  = calls[calls['Created Date'] < pd.Timestamp('20160101')].copy()

In [58]:
calls1015[:5]

,Created Date,Complaint Type,Incident Zip,Agency,Borough,City,Latitude,Longitude
0,2013-08-09 15:01:09,Building/Use,11238,DOB,BROOKLYN,BROOKLYN,40.6780042139,-73.9704404520
1,2013-08-09 14:44:28,Special Enforcement,10314,DOB,STATEN ISLAND,STATEN ISLAND,40.5914345421,-74.1647882378
2,2013-08-09 17:35:45,Building/Use,11364,DOB,QUEENS,Oakland Gardens,40.7476796746,-73.7594756840
3,2014-07-21 11:15:24,Blocked Driveway,10465,NYPD,BRONX,BRONX,40.8306933714,-73.8207504244
4,2013-08-09 22:42:00,Street Light Condition,11215,DOT,BROOKLYN,BROOKLYN,40.6722732893,-73.9857135722


In [59]:
calls1015['Created Date'].min()

Timestamp('2010-01-01 00:00:00')

In [60]:
calls1015['Created Date'].max()

Timestamp('2015-12-31 23:59:45')

In [61]:
calls1015['Year'] = pd.DatetimeIndex(calls1015['Created Date']).year
calls1015['Month'] = pd.DatetimeIndex(calls1015['Created Date']).month

In [62]:
calls1015[:3]

,Created Date,Complaint Type,Incident Zip,Agency,Borough,City,Latitude,Longitude,Year,Month
0,2013-08-09 15:01:09,Building/Use,11238,DOB,BROOKLYN,BROOKLYN,40.6780042139,-73.9704404520,2013,8
1,2013-08-09 14:44:28,Special Enforcement,10314,DOB,STATEN ISLAND,STATEN ISLAND,40.5914345421,-74.1647882378,2013,8
2,2013-08-09 17:35:45,Building/Use,11364,DOB,QUEENS,Oakland Gardens,40.7476796746,-73.7594756840,2013,8


In [74]:
complaints = calls1015['Complaint Type'].value_counts()
complaints

HEATING                              887869
Street Light Condition               597034
Street Condition                     588654
PLUMBING                             517486
GENERAL CONSTRUCTION                 500867
Blocked Driveway                     390839
Water System                         369653
PAINT - PLASTER                      361258
HEAT/HOT WATER                       358472
Illegal Parking                      286219
Traffic Signal Condition             278041
NONCONST                             260890
Noise                                243775
Sewer                                227390
ELECTRIC                             207873
Dirty Conditions                     206335
General Construction/Plumbing        179620
Damaged Tree                         177212
Noise - Commercial                   167785
Noise - Street/Sidewalk              166753
Sanitation Condition                 161606
Building/Use                         160655
UNSANITARY CONDITION            

In [75]:
complaints.shape

(247L,)

In [76]:
complaints[:25]

HEATING                          887869
Street Light Condition           597034
Street Condition                 588654
PLUMBING                         517486
GENERAL CONSTRUCTION             500867
Blocked Driveway                 390839
Water System                     369653
PAINT - PLASTER                  361258
HEAT/HOT WATER                   358472
Illegal Parking                  286219
Traffic Signal Condition         278041
NONCONST                         260890
Noise                            243775
Sewer                            227390
ELECTRIC                         207873
Dirty Conditions                 206335
General Construction/Plumbing    179620
Damaged Tree                     177212
Noise - Commercial               167785
Noise - Street/Sidewalk          166753
Sanitation Condition             161606
Building/Use                     160655
UNSANITARY CONDITION             144677
Rodent                           141196
Broken Muni Meter                138222


In [80]:
complaints[:25].sum() / float(complaints.sum())

0.7362934812536247

In [82]:
complaints[:25].index

Index([u'HEATING', u'Street Light Condition', u'Street Condition', u'PLUMBING',
       u'GENERAL CONSTRUCTION', u'Blocked Driveway', u'Water System',
       u'PAINT - PLASTER', u'HEAT/HOT WATER', u'Illegal Parking',
       u'Traffic Signal Condition', u'NONCONST', u'Noise', u'Sewer',
       u'ELECTRIC', u'Dirty Conditions', u'General Construction/Plumbing',
       u'Damaged Tree', u'Noise - Commercial', u'Noise - Street/Sidewalk',
       u'Sanitation Condition', u'Building/Use', u'UNSANITARY CONDITION',
       u'Rodent', u'Broken Muni Meter'],
      dtype='object')

In [94]:
calls1015['Complaint_Category'] = np.nan

In [95]:
calls1015[:3]

,Created Date,Complaint Type,Incident Zip,Agency,Borough,City,Latitude,Longitude,Year,Month,Complaint Category,Complaint_Category
0,2013-08-09 15:01:09,Building/Use,11238,DOB,BROOKLYN,BROOKLYN,40.6780042139,-73.9704404520,2013,8,NaN,NaN
1,2013-08-09 14:44:28,Special Enforcement,10314,DOB,STATEN ISLAND,STATEN ISLAND,40.5914345421,-74.1647882378,2013,8,NaN,NaN
2,2013-08-09 17:35:45,Building/Use,11364,DOB,QUEENS,Oakland Gardens,40.7476796746,-73.7594756840,2013,8,NaN,NaN


In [96]:
plumb_const_complaints = calls1015['Complaint Type'].isin( \
                        ['PLUMBING', 'GENERAL CONSTRUCTION', 'PAINT - PLASTER', 'NONCONST', \
                         'General Construction/Plumbing', 'Building/Use'] )

In [97]:
plumb_const_complaints[:3]

0     True
1    False
2     True
Name: Complaint Type, dtype: bool

In [98]:
calls1015.Complaint_Category[plumb_const_complaints] = 'Plumbing and Construction'

C:\Users\Mario\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [99]:
calls1015[:3]

,Created Date,Complaint Type,Incident Zip,Agency,Borough,City,Latitude,Longitude,Year,Month,Complaint Category,Complaint_Category
0,2013-08-09 15:01:09,Building/Use,11238,DOB,BROOKLYN,BROOKLYN,40.6780042139,-73.9704404520,2013,8,NaN,Plumbing and Construction
1,2013-08-09 14:44:28,Special Enforcement,10314,DOB,STATEN ISLAND,STATEN ISLAND,40.5914345421,-74.1647882378,2013,8,NaN,NaN
2,2013-08-09 17:35:45,Building/Use,11364,DOB,QUEENS,Oakland Gardens,40.7476796746,-73.7594756840,2013,8,NaN,Plumbing and Construction


In [100]:
heat_elec_complaints = calls1015['Complaint Type'].isin( ['HEATING', 'HEAT/HOT WATER', 'ELECTRIC'] )

In [101]:
street_traffic_parking_complaints = calls1015['Complaint Type'].isin(['Street Light Condition', \
                                    'Street Condition', 'Blocked Driveway', 'Illegal Parking', \
                                    'Traffic Signal Condition', 'Damaged Tree', 'Broken Muni Meter'] )

In [102]:
water_hygiene_complaints = calls1015['Complaint Type'].isin(['Water System', 'Sewer', 'Dirty Conditions', \
                                                             'Sanitation Condition', 'UNSANITARY CONDITION', 'Rodent'])

In [103]:
noise_complaints = calls1015['Complaint Type'].isin(['Noise', 'Noise - Commercial', 'Noise - Street/Sidewalk'])

In [104]:
calls1015.Complaint_Category[heat_elec_complaints] = 'Heating and Electric'

C:\Users\Mario\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [105]:
calls1015.Complaint_Category[street_traffic_parking_complaints] = 'Street, Traffic and Parking'

C:\Users\Mario\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [106]:
calls1015.Complaint_Category[water_hygiene_complaints] = 'Water and Hygiene'

C:\Users\Mario\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [107]:
calls1015.Complaint_Category[noise_complaints] = 'Noise'

C:\Users\Mario\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [108]:
calls1015[:3]

,Created Date,Complaint Type,Incident Zip,Agency,Borough,City,Latitude,Longitude,Year,Month,Complaint Category,Complaint_Category
0,2013-08-09 15:01:09,Building/Use,11238,DOB,BROOKLYN,BROOKLYN,40.6780042139,-73.9704404520,2013,8,NaN,Plumbing and Construction
1,2013-08-09 14:44:28,Special Enforcement,10314,DOB,STATEN ISLAND,STATEN ISLAND,40.5914345421,-74.1647882378,2013,8,NaN,NaN
2,2013-08-09 17:35:45,Building/Use,11364,DOB,QUEENS,Oakland Gardens,40.7476796746,-73.7594756840,2013,8,NaN,Plumbing and Construction


In [112]:
calls1015 = calls1015.drop(['Complaint Category'], axis=1)

In [113]:
calls1015[:3]

,Created Date,Complaint Type,Incident Zip,Agency,Borough,City,Latitude,Longitude,Year,Month,Complaint_Category
0,2013-08-09 15:01:09,Building/Use,11238,DOB,BROOKLYN,BROOKLYN,40.6780042139,-73.9704404520,2013,8,Plumbing and Construction
1,2013-08-09 14:44:28,Special Enforcement,10314,DOB,STATEN ISLAND,STATEN ISLAND,40.5914345421,-74.1647882378,2013,8,NaN
2,2013-08-09 17:35:45,Building/Use,11364,DOB,QUEENS,Oakland Gardens,40.7476796746,-73.7594756840,2013,8,Plumbing and Construction


In [114]:
calls1015[['Borough', 'Year', 'Month', 'Complaint_Category']][:3]

,Borough,Year,Month,Complaint_Category
0,BROOKLYN,2013,8,Plumbing and Construction
1,STATEN ISLAND,2013,8,NaN
2,QUEENS,2013,8,Plumbing and Construction


In [115]:
calls1015_pivdata = calls1015[['Borough', 'Year', 'Month', 'Complaint_Category']].copy()

In [116]:
calls1015_pivdata[:3]

,Borough,Year,Month,Complaint_Category
0,BROOKLYN,2013,8,Plumbing and Construction
1,STATEN ISLAND,2013,8,NaN
2,QUEENS,2013,8,Plumbing and Construction


In [122]:
calls1015_pivdata = calls1015_pivdata.dropna(subset=['Complaint_Category'])

In [123]:
calls1015_pivdata[:3]

,Borough,Year,Month,Complaint_Category
0,BROOKLYN,2013,8,Plumbing and Construction
2,QUEENS,2013,8,Plumbing and Construction
3,BRONX,2014,7,"Street, Traffic and Parking"


In [125]:
calls1015_pivdata['Var'] = 1

In [126]:
calls1015_pivdata[:3]

,Borough,Year,Month,Complaint_Category,Var
0,BROOKLYN,2013,8,Plumbing and Construction,1
2,QUEENS,2013,8,Plumbing and Construction,1
3,BRONX,2014,7,"Street, Traffic and Parking",1


In [138]:
calls1015_pivdata['Borough'].unique()

array(['BROOKLYN', 'QUEENS', 'BRONX', 'MANHATTAN', 'STATEN ISLAND',
       'Unspecified'], dtype=object)

In [144]:
calls1015_pivdata = calls1015_pivdata[calls1015_pivdata['Borough'] != 'Unspecified'].copy()

In [197]:
calls1015_pivdata.to_csv('calls1015_pivdata.csv')

In [151]:
pd.set_option('precision',0)

In [188]:
results_complaint_year = pd.pivot_table(calls1015_pivdata, index=[ 'Complaint_Category','Year'], values=['Var'], aggfunc=len, \
               columns=['Borough'])
results_complaint_year

Var                             \
Borough                            BRONX BROOKLYN MANHATTAN  QUEENS   
Complaint_Category          Year                                      
Heating and Electric        2010      37       48        20      16   
                            2011   44142    53023     32294   20863   
                            2012   66619    72382     47531   29086   
                            2013   77460    74708     53044   30065   
                            2014   88375    81238     55772   32523   
                            2015   80225    79133     59648   34020   
Noise                       2010    6811    17494     31054   11208   
                            2011    6131    17353     31954   10245   
                            2012    7176    20245     32843   11555   
                            2013    8118    24618     42214   12915   
                            2014   11368    34290     56974   18271   
                            2015   15093    43136     68452   22072   
Plumbing and Construction   2010    8678    24956     13488   27380   
                            2011  108165   150146     66634   60648   
                            2012  108689   142165     68353   59785   
                            2013  106678   139561     68382   55488   
                            2014   36201    54116     26906   31494   
                            2015   22973    36355     21147   24513   
Street, Traffic and Parking 2010   55426   112574     66696  129211   
                            2011   54245   110056     69214  120357   
                            2012   50096   105643     59071  121452   
                            2013   47667   113359     66195  111511   
                            2014   58637   140375     68660  140891   
                            2015   66777   160321     67873  164866   
Water and Hygiene           2010   31814    56195     29423   65710   
                            2011   29570    55469     26265   61549   
                            2012   26854    53487     24114   54299   
                            2013   24914    48762     25467   49558   
                            2014   43935    75929     39067   61172   
                            2015   54469    90817     48588   69781   

                                                
Borough                          STATEN ISLAND  
Complaint_Category          Year                
Heating and Electric        2010             2  
                            2011          2278  
                            2012          2754  
                            2013          2720  
                            2014          2859  
                            2015          2812  
Noise                       2010          2268  
                            2011          2032  
                            2012          2306  
                            2013          2426  
                            2014          2959  
                            2015          3385  
Plumbing and Construction   2010          4666  
                            2011         10947  
                            2012          9841  
                            2013         10003  
                            2014          5364  
                            2015          4066  
Street, Traffic and Parking 2010         29829  
                            2011         31984  
                            2012         26140  
                            2013         24628  
                            2014         32783  
                            2015         36499  
Water and Hygiene           2010         16327  
                            2011         17322  
                            2012         15125  
                            2013         15536  
                            2014         18014  
                            2015         18746

In [189]:
results_complaint_year.to_csv('results_complaint_year.csv')

In [190]:
results_complaint_borough = pd.pivot_table(calls1015_pivdata, index=[ 'Complaint_Category','Borough'], values=['Var'], aggfunc=len, \
               columns=['Year'])
results_complaint_borough

Var                          \
Year                                         2010    2011    2012    2013   
Complaint_Category          Borough                                         
Heating and Electric        BRONX              37   44142   66619   77460   
                            BROOKLYN           48   53023   72382   74708   
                            MANHATTAN          20   32294   47531   53044   
                            QUEENS             16   20863   29086   30065   
                            STATEN ISLAND       2    2278    2754    2720   
Noise                       BRONX            6811    6131    7176    8118   
                            BROOKLYN        17494   17353   20245   24618   
                            MANHATTAN       31054   31954   32843   42214   
                            QUEENS          11208   10245   11555   12915   
                            STATEN ISLAND    2268    2032    2306    2426   
Plumbing and Construction   BRONX            8678  108165  108689  106678   
                            BROOKLYN        24956  150146  142165  139561   
                            MANHATTAN       13488   66634   68353   68382   
                            QUEENS          27380   60648   59785   55488   
                            STATEN ISLAND    4666   10947    9841   10003   
Street, Traffic and Parking BRONX           55426   54245   50096   47667   
                            BROOKLYN       112574  110056  105643  113359   
                            MANHATTAN       66696   69214   59071   66195   
                            QUEENS         129211  120357  121452  111511   
                            STATEN ISLAND   29829   31984   26140   24628   
Water and Hygiene           BRONX           31814   29570   26854   24914   
                            BROOKLYN        56195   55469   53487   48762   
                            MANHATTAN       29423   26265   24114   25467   
                            QUEENS          65710   61549   54299   49558   
                            STATEN ISLAND   16327   17322   15125   15536   

                                                           
Year                                         2014    2015  
Complaint_Category          Borough                        
Heating and Electric        BRONX           88375   80225  
                            BROOKLYN        81238   79133  
                            MANHATTAN       55772   59648  
                            QUEENS          32523   34020  
                            STATEN ISLAND    2859    2812  
Noise                       BRONX           11368   15093  
                            BROOKLYN        34290   43136  
                            MANHATTAN       56974   68452  
                            QUEENS          18271   22072  
                            STATEN ISLAND    2959    3385  
Plumbing and Construction   BRONX           36201   22973  
                            BROOKLYN        54116   36355  
                            MANHATTAN       26906   21147  
                            QUEENS          31494   24513  
                            STATEN ISLAND    5364    4066  
Street, Traffic and Parking BRONX           58637   66777  
                            BROOKLYN       140375  160321  
                            MANHATTAN       68660   67873  
                            QUEENS         140891  164866  
                            STATEN ISLAND   32783   36499  
Water and Hygiene           BRONX           43935   54469  
                            BROOKLYN        75929   90817  
                            MANHATTAN       39067   48588  
                            QUEENS          61172   69781  
                            STATEN ISLAND   18014   18746

In [191]:
results_complaint_borough.to_csv('results_complaint_borough.csv')

In [255]:
results_borough_complaint = pd.pivot_table(calls1015_pivdata, index=[ 'Borough', 'Complaint_Category'], values=['Var'], aggfunc=len, \
               columns=['Year'])
results_borough_complaint

Var                          \
Year                                         2010    2011    2012    2013   
Borough       Complaint_Category                                            
BRONX         Heating and Electric             37   44142   66619   77460   
              Noise                          6811    6131    7176    8118   
              Plumbing and Construction      8678  108165  108689  106678   
              Street, Traffic and Parking   55426   54245   50096   47667   
              Water and Hygiene             31814   29570   26854   24914   
BROOKLYN      Heating and Electric             48   53023   72382   74708   
              Noise                         17494   17353   20245   24618   
              Plumbing and Construction     24956  150146  142165  139561   
              Street, Traffic and Parking  112574  110056  105643  113359   
              Water and Hygiene             56195   55469   53487   48762   
MANHATTAN     Heating and Electric             20   32294   47531   53044   
              Noise                         31054   31954   32843   42214   
              Plumbing and Construction     13488   66634   68353   68382   
              Street, Traffic and Parking   66696   69214   59071   66195   
              Water and Hygiene             29423   26265   24114   25467   
QUEENS        Heating and Electric             16   20863   29086   30065   
              Noise                         11208   10245   11555   12915   
              Plumbing and Construction     27380   60648   59785   55488   
              Street, Traffic and Parking  129211  120357  121452  111511   
              Water and Hygiene             65710   61549   54299   49558   
STATEN ISLAND Heating and Electric              2    2278    2754    2720   
              Noise                          2268    2032    2306    2426   
              Plumbing and Construction      4666   10947    9841   10003   
              Street, Traffic and Parking   29829   31984   26140   24628   
              Water and Hygiene             16327   17322   15125   15536   

                                                           
Year                                         2014    2015  
Borough       Complaint_Category                           
BRONX         Heating and Electric          88375   80225  
              Noise                         11368   15093  
              Plumbing and Construction     36201   22973  
              Street, Traffic and Parking   58637   66777  
              Water and Hygiene             43935   54469  
BROOKLYN      Heating and Electric          81238   79133  
              Noise                         34290   43136  
              Plumbing and Construction     54116   36355  
              Street, Traffic and Parking  140375  160321  
              Water and Hygiene             75929   90817  
MANHATTAN     Heating and Electric          55772   59648  
              Noise                         56974   68452  
              Plumbing and Construction     26906   21147  
              Street, Traffic and Parking   68660   67873  
              Water and Hygiene             39067   48588  
QUEENS        Heating and Electric          32523   34020  
              Noise                         18271   22072  
              Plumbing and Construction     31494   24513  
              Street, Traffic and Parking  140891  164866  
              Water and Hygiene             61172   69781  
STATEN ISLAND Heating and Electric           2859    2812  
              Noise                          2959    3385  
              Plumbing and Construction      5364    4066  
              Street, Traffic and Parking   32783   36499  
              Water and Hygiene             18014   18746

In [256]:
results_borough_complaint.to_csv('results_borough_complaint.csv')

In [192]:
results_complaint_year.ix['Heating and Electric']

Var                                        
Borough  BRONX BROOKLYN MANHATTAN QUEENS STATEN ISLAND
Year                                                  
2010        37       48        20     16             2
2011     44142    53023     32294  20863          2278
2012     66619    72382     47531  29086          2754
2013     77460    74708     53044  30065          2720
2014     88375    81238     55772  32523          2859
2015     80225    79133     59648  34020          2812

In [193]:
results_complaint_year.ix['Heating and Electric',2010]

     Borough      
Var  BRONX            37
     BROOKLYN         48
     MANHATTAN        20
     QUEENS           16
     STATEN ISLAND     2
Name: (Heating and Electric, 2010), dtype: int64

In [199]:
results_complaint_borough.ix['Heating and Electric', 'BRONX']

     Year
Var  2010       37
     2011    44142
     2012    66619
     2013    77460
     2014    88375
     2015    80225
Name: (Heating and Electric, BRONX), dtype: int64

In [201]:
#results_complaint_year.index.names

In [198]:
#results_complaint_year.query('Complaint_Category == "Heating and Electric"')

In [200]:
#results.stack().ix['Heating and Electric']

In [244]:
from bokeh.plotting import figure, show, vplot #,output_file
from bokeh.io import output_notebook
from bokeh.models import FixedTicker

In [219]:
output_notebook()

Loading BokehJS ...

In [254]:
# Complaint Type Based Plot

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select"

i=0
fig_num = ['fig1', 'fig2', 'fig3', 'fig4', 'fig5']
colors = ['FireBrick', 'DarkOrange', 'ForestGreen', 'RoyalBlue', 'Goldenrod']
x = np.linspace(2010, 2015, 6)

for title in ['Heating and Electric', 'Noise', 'Plumbing and Construction', \
             'Street, Traffic and Parking', 'Water and Hygiene']:
    
    fig_num[i] = figure(title=(title + ' Complaints'), tools=TOOLS, plot_width=750, plot_height=600)
    fig_num[i].xaxis.axis_label = "Year"
    fig_num[i].yaxis.axis_label = "# of Complaints"
    fig_num[i].xaxis[0].ticker=FixedTicker(ticks=[2010, 2011, 2012, 2013, 2014, 2015])
    fig_num[i].xaxis.bounds = (2010,2015)

    j=0
    
    for legend in ['BRONX', 'BROOKLYN', 'MANHATTAN', 'QUEENS', 'STATEN ISLAND']:
        
        y = results_complaint_borough.ix[title, legend]
        fig_num[i].circle(x, y, legend=legend, size=8, color=colors[j])
        fig_num[i].line(x, y, legend=legend, color=colors[j])
        j += 1
        
    fig_num[i].legend.location = "top_left"
    i += 1
    
#show(vplot(fig_num[0]))
show(vplot(fig_num[0], fig_num[1], fig_num[2], fig_num[3], fig_num[4])) 




In [259]:
# Borough Based Plot

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select"

i=0
fig_num = ['fig1', 'fig2', 'fig3', 'fig4', 'fig5']
colors = ['FireBrick', 'DarkOrange', 'ForestGreen', 'RoyalBlue', 'Goldenrod']
x = np.linspace(2010, 2015, 6)

for title in ['BRONX', 'BROOKLYN', 'MANHATTAN', 'QUEENS', 'STATEN ISLAND']:
    
    fig_num[i] = figure(title=('Complaints from ' + title), tools=TOOLS, plot_width=750, plot_height=600)
    fig_num[i].xaxis.axis_label = "Year"
    fig_num[i].yaxis.axis_label = "# of Complaints"
    fig_num[i].xaxis[0].ticker=FixedTicker(ticks=[2010, 2011, 2012, 2013, 2014, 2015])
    fig_num[i].xaxis.bounds = (2010,2015)

    j=0
    
    for legend in ['Heating and Electric', 'Noise', 'Plumbing and Construction', \
             'Street, Traffic and Parking', 'Water and Hygiene']:
        
        y = results_borough_complaint.ix[title, legend]
        fig_num[i].circle(x, y, legend=legend, size=8, color=colors[j])
        fig_num[i].line(x, y, legend=legend, color=colors[j])
        j += 1
        
    fig_num[i].legend.location = "top_left"
    i += 1
    
#show(vplot(fig_num[0]))
show(vplot(fig_num[0], fig_num[1], fig_num[2], fig_num[3], fig_num[4])) 